In [6]:
import pandas as pd
import numpy as np
import os

In [55]:
class NeuronPopRecording: #Suppose to be one file
    metadata_datanames=['noNeurons', 'dt', 'totalTimesteps', 'neuronPopId']
    def __init__(self, neuronPop_id : int):
        self.neuronPop_id=neuronPop_id
    def append_metadata(self, metadata_list : list):
        self.metadata=metadata_list
    def append_spiketimes(self, N_index : int, spiketimes_list : list):
        setattr(self, "Neuron_%s" % (N_index), spiketimes_list)
    def create_firing_matrix(self):
        self.F_matrix=np.zeros(shape=(self.metadata[2], self.metadata[0]))
        for neuron in dir(self):
            if neuron.startswith("Neuron"):
                neuron_index=int(neuron.split('_')[1])
                spiketimes=self.__dict__[neuron]
                for st in spiketimes:
                    self.F_matrix[st/self.metadata[1], neuron_index]=1
                

class OutputRecordingList: #Compendium of all recordings
    def get(self):
        return self.__dict__
    def append(self, var : NeuronPopRecording):
        setattr(self, "NeuronPop_%s" % (var.neuronPop_id), var)

#Here we use split = to get the name and the values, then split the second with commas to get the spike-time list
def read_all_output_files_into_object(neuronPop_files):
    all_recordingsST=OutputRecordingList()
    for i, file in enumerate(neuronPop_files):
        with open(file, "r") as file:
            lines=file.readlines()
            firstline=lines[0].rstrip() #The first line should be the metadata
            if not firstline.startswith('M'):
                print('Error with metadata reading')
                raise Exception
            metadata_list=firstline.split('=')[1].split(',')
            metadata_list[0]=int(metadata_list[0])
            metadata_list[1]=float(metadata_list[1])
            metadata_list[2]=int(metadata_list[2])
            metadata_list[3]=int(metadata_list[3])
            recording=NeuronPopRecording(metadata_list[3])
            recording.append_metadata(metadata_list)
            for line in lines:
                if line.startswith('M'):
                    continue
                elif line.startswith('N'):
                    line=line.split('=')
                    n_index=int(line[0].split('_')[1])
                    spiketimes=list()
                    for i in line[1].split(','):
                        if i=='':
                            continue
                        elif i=='\n':
                            continue
                        else:
                            spiketimes.append(float(i))
                    recording.append_spiketimes(n_index, spiketimes)
                else:
                    print('Oversight error')
                    raise Exception
        all_recordingsST.append(recording)
    return all_recordingsST

### File reading

In [56]:
#File reading here (files are always iterated on alphabetical order)
parsedOutput_string='_ParsedOutput.dat'

base_dir = os.getcwd()

neuronPop_files = [f for f in os.listdir(base_dir) if f.endswith(parsedOutput_string)]

### Reading parsed spike times file (Neuron output)
Always remember: the non-parsed files cannot be deleted by the programme. If they are very big it is recommended to delete them manually

In [57]:
all_recordings=read_all_output_files_into_object(neuronPop_files)
all_recordings.__dict__

{'NeuronPop_0': <__main__.NeuronPopRecording at 0x29b7602f280>,
 'NeuronPop_1': <__main__.NeuronPopRecording at 0x29b7602f340>}

In [58]:
dir(all_recordings.NeuronPop_0)
all_recordings.NeuronPop_0.__dict__

{'neuronPop_id': 0,
 'metadata': [10, 0.0001, 10000, 0],
 'Neuron_0': [0.0001,
  0.0011,
  0.0021,
  0.0031,
  0.0041,
  0.0051,
  0.0061,
  0.0071,
  0.0081,
  0.0091,
  0.0101,
  0.0111,
  0.0121,
  0.0131,
  0.0141,
  0.0151,
  0.0161,
  0.0171,
  0.0181,
  0.0191,
  0.0201,
  0.0211,
  0.0221,
  0.0231,
  0.0241,
  0.0251,
  0.0261,
  0.0271,
  0.0281,
  0.0291,
  0.0301,
  0.0311,
  0.0321,
  0.0331,
  0.0341,
  0.0351,
  0.0361,
  0.0371,
  0.0381,
  0.0391,
  0.0401,
  0.0411,
  0.0421,
  0.0431,
  0.0441,
  0.0451,
  0.0461,
  0.0471,
  0.0481,
  0.0491,
  0.0501,
  0.0511,
  0.0521,
  0.0531,
  0.0541,
  0.0551,
  0.0561,
  0.0571,
  0.0581,
  0.0591,
  0.0601,
  0.0611,
  0.0621,
  0.0631,
  0.0641,
  0.0651,
  0.0661,
  0.0671,
  0.0681,
  0.0691,
  0.0701,
  0.0711,
  0.0721,
  0.0731,
  0.0741,
  0.0751,
  0.0761,
  0.0771,
  0.0781,
  0.0791,
  0.0801,
  0.0811,
  0.0821,
  0.0831,
  0.0841,
  0.0851,
  0.0861,
  0.0871,
  0.0881,
  0.0891,
  0.0901,
  0.0911,
  0.0921,
 